In [166]:
# %%pycodestyle


In [167]:
import random

from deap import base
from deap import creator
from deap import tools

from PIL import Image
from PIL import ImageChops
import numpy as np
import math
import operator

from deap import base, creator, tools, algorithms
from random import randint, random, gauss
from PIL import Image, ImageDraw
from functools import partial
from math import sqrt
import numpy
import random
import pickle
import pandas as pd
import os.path


In [168]:

PIC = Image.open('template.jpg')
SIZE = 100
VALUES_PER_GENE = 13

NUMBER_OF_TRIANGLES = 3
POPULATION = 4
NGEN = 1000
CXPB = .5
MUTPB = .1

FREQ = 2

NUM_OF_DRAW_OPTIONS = len(["circle", "triangle", "line", "rectangle"])
MAX_CIR_RADIUS = int(SIZE * .1)
LINE_LENGTH = int(SIZE * .15)
LINE_WIDTH = int(SIZE*.01)

MUTATION_NUM_TO_RANDOM = 5

In [169]:


def computeSimilarity(allInds):
    numpyInds = numpy.array(allInds)
    groupedGenesShape = (NUMBER_OF_TRIANGLES*VALUES_PER_GENE, POPULATION)
    groupedGenes = np.reshape(numpyInds.ravel(order='F'), groupedGeneShape)
    numpy.ndarray.sort(groupedGenes, axis=1)

    diffs = numpy.apply_along_axis(np.diff, 1, groupedGenes)

    multipliers = [i * (POPULATION-i) for i in range(POPULATION)]
    multipliers = multipliers[1:]

    scaledDiffs = numpy.apply_along_axis(lambda x: x*multipliers, 1, diffs)
    averageDiffInGene = numpy.apply_along_axis(numpy.mean, 1, diffs)

    return 1-numpy.mean(averageDiffInGene)

In [170]:


def gen_one_triangle():
    return [random.random() for i in range(VALUES_PER_GENE)]

In [171]:


def drawImage(triangles):
    im = Image.new('RGB', (SIZE, SIZE), (255, 255, 255))
    for tri in triangles:
        mask = Image.new('RGBA', (SIZE, SIZE))
        draw = ImageDraw.Draw(mask)

        drawShape = int(tri[0] * NUM_OF_DRAW_OPTIONS)
        x1 = int(tri[1] * SIZE)
        y1 = int(tri[2] * SIZE)
        x2 = int(tri[3] * SIZE)
        y2 = int(tri[4] * SIZE)
        x3 = int(tri[5] * SIZE)
        y3 = int(tri[6] * SIZE)
        r = int(tri[7] * 255)
        g = int(tri[8] * 255)
        b = int(tri[9] * 255)
        alpha = int(tri[10] * 40)
        radius = int(tri[11] * MAX_CIR_RADIUS)
        direction = math.radians(int(tri[12]*360))

        fillColor = (r, g, b, alpha)

        # triangle
        if drawShape == 0:
            triangle = ((x1, y1), (x2, y2), (x3, y3))
            fillColor = (r, g, b, alpha)
            draw.polygon(triangle, fill=fillColor)

        # circle
        if drawShape == 1:
            lowerX, upperX = x1-radius, x1+radius
            lowerY, upperY = y1-radius, y1+radius
            fillColor = (r, g, b, alpha)
            draw.ellipse((lowerX, lowerY, upperX, upperY), fill=fillColor)

        # line
        if drawShape == 2:
            x2 = x1 + math.cos(direction) * LINE_LENGTH
            y2 = y1 + math.sin(direction) * LINE_LENGTH
            draw.line([x1, y1, x2, y2], width=LINE_WIDTH, fill=fillColor)

        # rectangle
        if drawShape == 3:
            draw.rectangle([x1, y1, x2, y2], fill=fillColor)

        im.paste(mask, mask=mask)
        del mask, draw
    return im


In [172]:


def evaluate(im1, t2):
    im2 = drawImage(t2)

    h = ImageChops.difference(im1, im2).histogram()
    length = int(len(h)/3)

    r = h[length*0:length*1]
    g = h[length*1:length*2]
    b = h[length*2:length*3]

    h = [sum(x) for x in zip(r, g, b)]
    errSqrd = sum(h*(i**2) for i, h in enumerate(h)) / float(SIZE * SIZE * 3)
    err = math.sqrt(errSqrd) / 256
    return 1 - err,

In [173]:


def mutate(triangles):
    for i in range(MUTATION_NUM_TO_RANDOM):
        triangleToMutate = triangles[randint(0, len(triangles)-1)]
        valueIndexToMutate = randint(0, len(triangleToMutate)-1)
        triangleToMutate[valueIndexToMutate] = random.random()

    return triangles,

In [174]:


creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

toolbox = base.Toolbox()
toolbox.register("attr", gen_one_triangle)
toolbox.register("individual", tools.initRepeat,
                 creator.Individual, toolbox.attr, NUMBER_OF_TRIANGLES)
toolbox.register("population", tools.initRepeat,
                 list, toolbox.individual)

toolbox.register("evaluate", partial(evaluate, PIC))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

logbook = None

C:\Users\Joe\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Joe\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [175]:


def main(checkpoint=None):
    if checkpoint and os.path.isfile(checkpoint):
        # A file name has been given, then load the data from the file
        with open(checkpoint, "rb") as cp_file:
            cp = pickle.load(cp_file)
        pop = cp["population"]
        start_gen = cp["generation"]
        hof = cp["halloffame"]
        logbook = cp["logbook"]
        random.setstate(cp["rndstate"])
    else:
        # Start a new evolution
        pop = toolbox.population(n=POPULATION)
        start_gen = 0
        hof = tools.HallOfFame(maxsize=1)
        logbook = tools.Logbook()

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_sim = tools.Statistics(lambda ind: ind)
    stats_fit.register("std", numpy.std)
    stats_fit.register("max", numpy.max)
    stats_fit.register("avg", numpy.mean)
    stats_fit.register("min", numpy.min)
    stats_sim.register("similarity", computeSimilarity)

    mstats = tools.MultiStatistics(fitness=stats_fit, similarity=stats_sim)

    log = ""
    try:
        for gen in range(start_gen, NGEN):
            pop = algorithms.varAnd(pop, toolbox, cxpb=CXPB, mutpb=MUTPB)

            # Evaluate the individuals with an invalid fitness
            invalid_ind = [ind for ind in pop if not ind.fitness.valid]
            fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit

            hof.update(pop)
            record = mstats.compile(pop)
            logbook.record(gen=gen, evals=len(invalid_ind), **record)
            print(logbook.stream)

            pop = toolbox.select(pop, k=len(pop))

            if gen % FREQ == 0:
                # Fill the dictionary using the dict(key=value[, ...])
                # constructor
                cp = dict(population=pop, generation=gen, halloffame=hof,
                          logbook=logbook, rndstate=random.getstate())

                with open("checkpoint.pkl", "wb") as cp_file:
                    pickle.dump(cp, cp_file)
    finally:
        df_log = pd.DataFrame(logbook)
        df_log.to_csv('stats.txt', index=False)
        open('result.txt', 'w').write(repr(hof[0]))
        drawImage(hof[0]).save('result.png')


# if __name__ == '__main__':
    # main("checkpoint.pkl")
    # main()

In [176]:


import cProfile
cProfile.run('main("checkpoint.pkl")')

         5722 function calls (5395 primitive calls) in 0.019 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:995(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 <ipython-input-169-258d091bdc9b>:3(computeSimilarity)
       12    0.000    0.000    0.000    0.000 <ipython-input-170-8b036e2730c0>:3(gen_one_triangle)
       12    0.000    0.000    0.000    0.000 <ipython-input-170-8b036e2730c0>:4(<listcomp>)
        5    0.000    0.000    0.004    0.001 <ipython-input-171-c550ac4fd589>:3(drawImage)
        4    0.000    0.000    0.000    0.000 <ipython-input-172-1e95be189c47>:13(<listcomp>)
     1028    0.001    0.000    0.001    0.000 <ipython-input-172-1e95be189c47>:14(<genexpr>)
        4    0.000    0.000    0.005    0.001 <ipython-input-172-1e95be189c47>:3(evaluate)
        4    0.000    0.000    0.000    0.000 <ipython-input-175-af5

NameError: name 'groupedGeneShape' is not defined